$$
\textbf{GRPO:} \quad
\frac{1}{G} \sum_{i=1}^{G} \textcolor{red}{\frac{1}{|\mathbf{o}_i|}} \sum_{t=1}^{|\mathbf{o}_i|} 
\left\{ 
    \min \left[
        \frac{\pi_\theta(o_{i,t} \mid \mathbf{q}, \mathbf{o}_{i,<t})}{\pi_{\theta_{\text{old}}}(o_{i,t} \mid \mathbf{q}, \mathbf{o}_{i,<t})} \hat{A}_{i,t},
        \mathrm{clip} \left( 
            \frac{\pi_\theta(o_{i,t} \mid \mathbf{q}, \mathbf{o}_{i,<t})}{\pi_{\theta_{\text{old}}}(o_{i,t} \mid \mathbf{q}, \mathbf{o}_{i,<t})},
            1 - \epsilon, 1 + \epsilon
        \right) \hat{A}_{i,t}
    \right]
\right\}
$$

$$
\text{where} \quad
\hat{A}_{i,t} = 
\frac{
R(\mathbf{q}, \mathbf{o}_i) - \mathrm{mean}(\{ R(\mathbf{q}, \mathbf{o}_1), \ldots, R(\mathbf{q}, \mathbf{o}_G) \})
}{
\textcolor{red}{
\mathrm{std}(\{ R(\mathbf{q}, \mathbf{o}_1), \ldots, R(\mathbf{q}, \mathbf{o}_G) \})
}
}
$$

---

$$
\textbf{Dr. GRPO:} \quad \text{GRPO Done Right (without bias)} 
$$

$$
\frac{1}{G} \sum_{i=1}^{G} \sum_{t=1}^{|\mathbf{o}_i|} 
\left\{ 
    \min \left[
        \frac{\pi_\theta(o_{i,t} \mid \mathbf{q}, \mathbf{o}_{i,<t})}{\pi_{\theta_{\text{old}}}(o_{i,t} \mid \mathbf{q}, \mathbf{o}_{i,<t})} \hat{A}_{i,t},
        \mathrm{clip} \left( 
            \frac{\pi_\theta(o_{i,t} \mid \mathbf{q}, \mathbf{o}_{i,<t})}{\pi_{\theta_{\text{old}}}(o_{i,t} \mid \mathbf{q}, \mathbf{o}_{i,<t})},
            1 - \epsilon, 1 + \epsilon
        \right) \hat{A}_{i,t}
    \right]
\right\}
$$

$$
\text{where} \quad
\hat{A}_{i,t} = R(\mathbf{q}, \mathbf{o}_i) - \mathrm{mean}(\{ R(\mathbf{q}, \mathbf{o}_1), \ldots, R(\mathbf{q}, \mathbf{o}_G) \})
$$


- Dr. GRPO
    - https://arxiv.org/pdf/2503.20783
    - 移除了内部的平均，计算 advantage 不需要除以 std

### basics

- steps
    - effective_batch_size: per_device_train_batch_size * num_processes * gradient_accumulation_steps
        - 8 * 2 * 4
    - batch_size = effective_batch_size / num_generations = 64 / 8 = 8
    - steps: len * epochs / batch_size = 200 * 3 / 8 = 75
- `_get_train_sampler`
    - num_processes: 3, gpu_0, gpu_1, gpu_2
    - per_device_train_batch_size: 4
    - grad_accum: 3
    - num_generations: 3
    - batch_size = 4 * 3 * 3 / 3 = 12
- `scale_rewards`
    - Whether to scale the rewards by dividing them by their standard deviation. If `True` (default), the rewards are normalized by the standard deviation, ensuring they have unit variance. If `False`, no scaling is applied.
    - The [Dr. GRPO](https://github.com/sail-sg/understand-r1-zero/blob/main/understand-r1-zero.pdf) paper recommends not scaling the rewards, as scaling by the standard deviation introduces a question-level difficulty bias.

## `_prepare_inputs` & `compute_loss`

- _prepare_inputs
    - prompt_completion_ids: prompt + completion
- $-\beta D_{kl}[\pi_\theta\|\pi_{ref}]$
    - beta: 0.04 (default)
    - $\log \frac{\pi_{ref}}{\pi_\theta}$ 数值变得有些大时，`exp(log_ratio)` 会指数级爆炸；

```python
if self.beta != 0.0:
    ref_per_token_logps = inputs["ref_per_token_logps"]
    per_token_kl = (
        torch.exp(ref_per_token_logps - per_token_logps) - (ref_per_token_logps - per_token_logps) - 1
    )
```

### token-level loss

- https://github.com/huggingface/trl/blob/v0.15.2/trl/trainer/grpo_trainer.py#L719
    - per-sequence normalization
- https://github.com/huggingface/trl/blob/main/trl/trainer/grpo_trainer.py#L961
    - global-norm(token-level)
    - each unmasked token's loss provides the same contribution to the total loss.
- related
    - https://github.com/huggingface/trl/pull/2881/files
    - https://x.com/zzlccc/status/1904907968417497202

In [1]:
import torch

In [3]:
per_token_loss = torch.tensor([[1, 2], 
                               [3, 4], 
                               [5, 6]])
completion_mask = torch.tensor([[1, 1], 
                                [1, 0], 
                                [1, 0]])

In [4]:
# ppo(rlhf) loss
grpo_loss = ((per_token_loss * completion_mask).sum(dim=1) / completion_mask.sum(dim=1)).mean()
grpo_loss

tensor(3.1667)

In [5]:
dr_grpo_loss = (per_token_loss * completion_mask).sum() / completion_mask.sum()
dr_grpo_loss

tensor(2.7500)

In [9]:
# mean(row_sum(loss * mask)/row_sum(mask)) -> 906
# gets a higher loss vs
# sum(loss * mask)/sum(mask) -> 727

# extremely imbalanced losses and extremely imbalanced completion lengths
per_token_loss = torch.tensor([[1000, 1, 1, 1], 
                               [1000, 1, 1, 1], 
                               [1000, 1, 1, 1],
                               [1000, 1, 1, 1],
                               [1000, 1, 1, 1], 
                               [1000, 1, 1, 1], 
                               [1000, 1, 1, 1],
                               [1000, 1, 1, 1]])
completion_mask = torch.tensor([[1, 1, 1, 1], 
                                [1, 0, 0, 0], 
                                [1, 0, 0, 0], 
                                [1, 0, 0, 0], 
                                [1, 0, 0, 0], 
                                [1, 0, 0, 0], 
                                [1, 0, 0, 0], 
                                [1, 0, 0, 0]])

In [7]:
grpo_loss = ((per_token_loss * completion_mask).sum(dim=1) / completion_mask.sum(dim=1)).mean()
grpo_loss

tensor(906.3438)

In [8]:
dr_grpo_loss = (per_token_loss * completion_mask).sum() / completion_mask.sum()
dr_grpo_loss

tensor(727.5455)

In [14]:
per_token_loss = torch.ones_like(torch.empty(8, 4096))
per_token_loss[:, 0] = 1000
# per_token_loss
completion_mask = torch.zeros_like(torch.empty(8, 4096))
completion_mask[0, :] = 1
completion_mask[:, 0] = 1
completion_mask

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [15]:
grpo_loss = ((per_token_loss * completion_mask).sum(dim=1) / completion_mask.sum(dim=1)).mean()
grpo_loss

tensor(875.1555)

In [16]:
(per_token_loss * completion_mask).sum() / completion_mask.sum()

tensor(2.9478)

### verl

- 默认开启的就是 token-level loss （global mean）

    ```python
    
    def masked_mean(values, mask, axis=None):
        """Compute mean of tensor with a masked values."""
        return (values * mask).sum(axis=axis) / mask.sum(axis=axis)
    
    def compute_policy_loss(old_log_prob, log_prob, advantages, eos_mask, cliprange):
        """Adapted from https://github.com/huggingface/trl/blob/main/trl/trainer/ppo_trainer.py#L1122
    
        Args:
            old_log_prob: `(torch.Tensor)`
                shape: (bs, response_length)
            log_prob: `(torch.Tensor)`
                shape: (bs, response_length)
            advantages: `(torch.Tensor)`
                shape: (bs, response_length)
            eos_mask: `(torch.Tensor)`
                shape: (bs, response_length)
            cliprange: (float)
                The clip range used in PPO. See https://arxiv.org/abs/1707.06347
    
        Returns:
            pg_loss: `a scalar torch.Tensor`
                policy gradient loss computed via PPO
            pg_clipfrac: (float)
                a float number indicating the fraction of policy gradient loss being clipped
    
        """
        negative_approx_kl = log_prob - old_log_prob
        ratio = torch.exp(negative_approx_kl)
        ppo_kl = verl_F.masked_mean(-negative_approx_kl, eos_mask)
    
        pg_losses = -advantages * ratio
        pg_losses2 = -advantages * torch.clamp(ratio, 1.0 - cliprange, 1.0 + cliprange)
    
        pg_loss = verl_F.masked_mean(torch.max(pg_losses, pg_losses2), eos_mask)
        pg_clipfrac = verl_F.masked_mean(torch.gt(pg_losses2, pg_losses).float(), eos_mask)
        return pg_loss, pg_clipfrac, ppo_kl
    ```

## MISC

- wandb
    - `report_to='wandb'`
    - 默认的 wandb project 为 huggingface
        - `export WANDB_PROJECT=my_prpject_name`
- use_vllm